In [0]:

from pyspark.sql.functions import count, avg
# print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_not_join_20200218_20200223_2/").show() #.groupBy("")
# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_not_join_20200218_20200223_2/").filter("process_date='2020-02-22'").show()  # groupBy("process_date").agg({"process_date":"count"}).show()
# 

# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_not_join_20200218_20200221_1/").groupBy("event_month").agg({"event_month":"count"}).show()
# df1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_not_join_20200218_20200223_2/")
# df2 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/")
df2 = df2.drop("process_date")
df3 = df1.join(df2, "review_id")
# df3.show()
df3.groupBy("product_id", "process_date").agg({"product_id":"count","process_date":"count"}).orderBy("process_date").show()

# df3.filter("product_id='1475474300'").show()

In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/



In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
def citus_row():
    def get_data_in_citus():
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_citus_db",
                user="citus_bdp_usage_qa",
                host="10.2.10.132",
                password="dNzWtSV3pKTx",
                port=5432
            )
        )
        sql = " select review_id from advancedreview.advancedreview_topic_fact_v3 where _identifier ='120200730080215828' and country_code='JP' "
        db_data = query(citus_dsn_, sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus()
    return [Row(review_id=r[0] ) for r in result]
df = citus_row()


In [0]:

df_db = spark.createDataFrame(df)
df_db.createOrReplaceTempView("db")


df1 =  spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/").where("process_date='2020-07-29'").cache()
df1.createOrReplaceTempView("temp")
# spark.sql("select date, device_code, language, product_id, review_id, count(review_id) as review_id_count from temp where country_code='JP' group by date, device_code, language, product_id, review_id order by review_id_count desc").show()
# spark.sql("select  _identifier, country_code, count(1) from temp group by country_code, _identifier").show()

spark.sql("select * from temp where review_id in (select review_id from temp where country_code='JP' except all select * from db)").show()
spark.sql("select review_id from temp where review_id in (select review_id from temp where country_code='JP' except all select * from db)").show()

# spark.sql("select * from db except all select review_id from temp where country_code='JP'").show()

spark.sql("select * from temp where country_code='JP' and product_id=1064917658 ").show()
spark.sql("select * from temp where country_code='JP' and review_id=6250300160 ").show()




# spark.sql("select * from temp where review_id = 6250300160").show()

In [0]:

spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/").where("process_date = '2020-07-29'").createOrReplaceTempView("adv_data")
spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/").where("process_date = '2020-07-29'").createOrReplaceTempView("review_data")
spark.sql("""
select * from adv_data a left join review_data rv
on a.review_id = rv.review_id 
where rv.review_id is null
""").show()

In [0]:

df3= spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact").where("process_date='2020-03-07' and process_hour='23'").filter("review_id='5623338514'")
df3.show()

path="process_granularity={hourly,daily,weekly}/process_date=2020-03-07/process_hour=23"
spark.read.option('basePath','s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/').parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v4/fact/{}".format(path)).where("market_code in ('apple-store','google-play') and device_code in ('android-all', 'ios-all')").filter("review_id='5623338514'").show()


In [0]:


df1 =  spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_not_join/process_date=2020-03-07/")
df2 =  spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/").where("process_date='2020-03-07'")

df1.createOrReplaceTempView("not_join")
df2.createOrReplaceTempView("topic")

spark.sql("select * from topic where review_id = '5623338514' ").show()
spark.sql("select not_join.review_id,topic.product_id  from not_join inner join topic on not_join.review_id = topic.review_id ").show()



In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_not_join/ 


In [0]:

# df1=spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.topic.v1/fact/").where("review_id='5549512716' ").show()
print spark.read.format("delta").parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/").createOrReplaceTempView("topic")
spark.sql("select distinct Cast(_identifier AS long) as number_identifer from topic where _identifier like '%12020061%' order by number_identifer desc").show()

In [0]:

import unittest
import boto3
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3 
from aadatapipelinecore.core.fs import Conf
import json
import time
from datetime import datetime
from elasticsearch import Elasticsearch
import json
import pyspark.sql.functions as F
import json
import time
from datetime import datetime
from elasticsearch import Elasticsearch
import json



s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-advancedreview'))
path_list= s3_bucket_list.all(prefix="unified/aa.old_es_0211_before_2/", depth_is_1=True )
print path_list[0].split("=")[1].replace("-","").strip("/")

samples=[]
def test_detail_of_unified_db(path_list):
    for path in path_list:
        print path
        unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/{}".format(path)).sample(False, 0.1, seed=0)
        unified_df.registerTempTable("advancedreviews")

        df = spark.sql("select * from advancedreviews limit 10")
        
        rows = (df
        # .withColumn("date", F.date_format("time", "yyyy-MM-dd"))
        .withColumn("chinese_content", F.when((df.content_language == "zh") | (df.content_language == "zh_Hant"), df.content))
        .withColumn("chinese_reply", F.when((df.reply_language == "zh") | (df.reply_language == "zh_Hant"), df.reply))
        .withColumn("chinese_title", F.when((df.title_language == "zh") | (df.title_language == "zh_Hant"), df.title))
        .withColumn("english_content", F.when(df.content_language == "en", df.content))
        .withColumn("english_reply", F.when(df.reply_language == "en", df.reply))
        .withColumn("english_title", F.when(df.title_language == "en", df.title))
        .withColumn("japanese_content", F.when(df.content_language == "ja", df.content))
        .withColumn("japanese_reply", F.when(df.reply_language == "ja", df.reply))
        .withColumn("japanese_title", F.when(df.title_language == "ja", df.title))
        .withColumn("korean_content", F.when(df.content_language == "ko", df.content))
        .withColumn("korean_reply", F.when(df.reply_language == "ko", df.reply))
        .withColumn("korean_title", F.when(df.title_language == "ko", df.title))
        .withColumn("other_language_content", F.when(
            (df.content_language != "zh") & (df.content_language != "zh_Hant") & (df.content_language != "en") &
            (df.content_language != "ja") & (df.content_language != "ko"), df.content))
        .withColumn("other_language_reply", F.when(
            (df.reply_language != "zh") & (df.reply_language != "zh_Hant") & (df.reply_language != "en") &
            (df.reply_language != "ja") & (df.reply_language != "ko"), df.reply))
        .withColumn("other_language_title", F.when(
            (df.title_language != "zh") & (df.title_language != "zh_Hant") & (df.title_language != "en") &
            (df.title_language != "ja") & (df.title_language != "ko"), df.title))
        .collect()
        )
        print 'get several data',len(rows)
        samples.append(rows)
    return samples
    print 'sample size is : {}'.format(len(sample))




def es_doc(review_id):
    es_connection = Elasticsearch(["http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200"],http_auth=("bdp","C38vEJEuraCw"),port=19200)
    doc = es_connection.search(index="int-ss-advancedreview_v1-apple-store-ios-all*", body={"query":{"match":{"_id":"{}".format(review_id)}}})
    return doc



def compare(doc, row):
    keys = [ u'_identifier', u'app_id', u'chinese_content', u'chinese_reply', u'chinese_title', u'content', u'content_language', u'country_code', u'date', u'device_code', u'english_content', u'english_reply', u'english_title', u'japanese_content', u'japanese_reply', u'japanese_title', u'korean_content', u'korean_reply', u'korean_title', u'language', u'market_code', u'other_language_content', u'other_language_reply', u'other_language_title', u'product_version', u'rating', u'reply', u'reply_date', u'reply_language', u'title', u'title_language', u'user_device', u'user_id', u'user_language', u'user_name', u'user_purchased', u'topic_ids']
    row_dict =  row
    print 'row dict', row_dict
     
    doc = doc["hits"]["hits"][0]["_source"]
    print doc
    for key in keys:
        if key in ("app_id", "_identifier"):
            assert str(doc[key]) == str(row_dict[key]), "{} is diffrent".format(key)
        else:
            assert doc[key] == row_dict[key], "{} is diffrent".format(key)
# test_detail_of_unified_db(path_list)
for sample in samples[1]:
    print sample
    doc = es_doc(sample.review_id)
    compare(doc, sample)
    print 'pass'

In [0]:

# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/")
df1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.topic.v1/fact/").select("review_id")#.drop("doc_id").withColumn("topic_ids", cast(ArrayType(DataTypes.IntegerType)))
df2 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact").select("review_id")

# df3 = df1.union(df2)
# print df3.select("review_id").distinct().count()

# print df1.columns
# print df2.columns


In [0]:

df3 = df1.union(df2)
print df3.distinct().count()


In [0]:
%%sh
curl -XGET -u bdp:C38vEJEuraCw  -H 'Content-Type: application/json'   http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/int-ss-advancedreview_v1*/_count?



In [0]:
%python
16060027-16059966

In [0]:


print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_not_join/").filter("process_date='2020-02-25'").show()


In [0]:

from elasticsearch import Elasticsearch
import json
# review_miss_list = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_not_join_201908_20200211_4/").groupBy("process_date").agg({"process_date":"count"}).show()
# # print review_miss_list
# l = []
# for x in review_miss_list:
#     l.append(str(x.review_id))
# print l

def es_doc(review_id):
    es_connection = Elasticsearch(["http://internal-aa-int-ss-review-elasticsearch-126620990.us-east-1.elb.amazonaws.com:19200/"],http_auth=("bdp_read","E9vjhU2qG1bAcM83"),port=19200)
    doc = es_connection.search(index="int-reviews-*", body={"query":{"match":{"_id":"{}".format(review_id)}}})
    return doc
    
print es_doc("5550015704")

In [0]:

from elasticsearch import Elasticsearch
import json

from pyspark.sql.functions import count, avg

# review_miss_list = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.old_es_0211_before_2/event_month={2019-10,2019-11,2019-12,2020-01}/").collect()
review_miss_list = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_not_join/process_date=2020-03-08").collect()

# print review_miss_list
l = []
for x in review_miss_list:
    l.append(str(x.review_id))
print len(l)

not_find = []
def es_doc(review_id):
    # print 'review_id ', review_id
    es_connection = Elasticsearch(["http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200"],http_auth=("bdp","C38vEJEuraCw"),port=19200)
    doc = es_connection.search(index="int-ss-advancedreview_v1-apple-store-ios-all-202003*", body={"query":{"match":{"_id":"{}".format(review_id)}}})
    if len(doc["hits"]["hits"]) < 1:
        not_find.append(review_id)
    # return doc
    
print 'start to test es'
for review_id in l:
    es_doc(review_id)
print not_find
# 2020-03-07 - ['5583818364']
# 2020-03-08- ['5513989363', '5544648603', '5587852245', '5513608208', '5549250577', '5591995058', '5513186983', '5535403661', '5584841050', '5586886540', '5531232282', '5580175938', '5498922564', '5591153214', '5597883095', '5515830987', '5589648055', '5565223614', '5531491918', '5571782488', '5573295267', '5549059820']



In [0]:

import unittest
import boto3
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3 
from aadatapipelinecore.core.fs import Conf
import json
import time
from datetime import datetime
from elasticsearch import Elasticsearch
import json


# get up level data set

# s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-advancedreview'))
# # path_list= s3_bucket_list.all(prefix="unified/advancedreview.topic.v1/", depth_is_1=True )
path_list=[1,2]#= s3_bucket_list.all(prefix="unified/advancedreview.topic.v1/fact/", depth_is_1=True )



print path_list.pop(0)
# # print path_list[0].split("=")[-1].strip("/")
def get_identifer_count_list():
    count_list=[]
    for d in path_list:
        process_date = d.split("=")[-1].strip("/")
        identifer_count_list = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/process_date={}/".format(process_date)).dropDuplicates(["review_id"]).groupBy("_identifier").agg({"*":"count"}).collect()
        print identifer_count_list
        count_list.extend(identifer_count_list)
    return count_list
    
def es_doc(_identifer):
    # print 'review_id ', review_id
    es_connection = Elasticsearch(["http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200"],http_auth=("bdp","C38vEJEuraCw"),port=19200)
    doc = es_connection.count(index="int-ss-advancedreview_v1-apple-store-ios-all-*", body={"query":{"match":{"_identifier":"{}".format(_identifer)}}})
    return doc["count"]
    
count_list = get_identifer_count_list()
print 'count_list length is : ', len(count_list)
# print count_list[0]["_identifier"]
print len(count_list)
for r in count_list :
    # print r
    ident =  r["_identifier"]
    count =  r["count(1)"]
    db_count = es_doc(ident)
    print db_count, count
    if db_count != count:
        print "db count is {}, unified count: {}, diff is : {}, identiferis {} ".format(db_count, count, db_count- count, r)



In [0]:

import unittest
import boto3
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3 
from aadatapipelinecore.core.fs import Conf
import json
import time
from datetime import datetime
from elasticsearch import Elasticsearch
import json
import pyspark.sql.functions as F
import json
import time
from datetime import datetime
from elasticsearch import Elasticsearch
import json


# get up level data set
s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-advancedreview'))
path_list= s3_bucket_list.all(prefix="unified/aa.adv_join_201908_20200211_2/", depth_is_1=True )
print path_list[0].split("=")[1].replace("-","").strip("/")

samples=[]
def test_detail_of_unified_db(path_list):
    for path in path_list:
        print path
        unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/{}".format(path)).sample(False, 0.1, seed=0)
        unified_df.registerTempTable("advancedreviews")

        df = spark.sql("select * from advancedreviews limit 10")
        
        rows = (df
        # .withColumn("date", F.date_format("time", "yyyy-MM-dd"))
        .withColumn("chinese_content", F.when((df.content_language == "zh") | (df.content_language == "zh_Hant"), df.content))
        .withColumn("chinese_reply", F.when((df.reply_language == "zh") | (df.reply_language == "zh_Hant"), df.reply))
        .withColumn("chinese_title", F.when((df.title_language == "zh") | (df.title_language == "zh_Hant"), df.title))
        .withColumn("english_content", F.when(df.content_language == "en", df.content))
        .withColumn("english_reply", F.when(df.reply_language == "en", df.reply))
        .withColumn("english_title", F.when(df.title_language == "en", df.title))
        .withColumn("japanese_content", F.when(df.content_language == "ja", df.content))
        .withColumn("japanese_reply", F.when(df.reply_language == "ja", df.reply))
        .withColumn("japanese_title", F.when(df.title_language == "ja", df.title))
        .withColumn("korean_content", F.when(df.content_language == "ko", df.content))
        .withColumn("korean_reply", F.when(df.reply_language == "ko", df.reply))
        .withColumn("korean_title", F.when(df.title_language == "ko", df.title))
        .withColumn("other_language_content", F.when(
            (df.content_language != "zh") & (df.content_language != "zh_Hant") & (df.content_language != "en") &
            (df.content_language != "ja") & (df.content_language != "ko"), df.content))
        .withColumn("other_language_reply", F.when(
            (df.reply_language != "zh") & (df.reply_language != "zh_Hant") & (df.reply_language != "en") &
            (df.reply_language != "ja") & (df.reply_language != "ko"), df.reply))
        .withColumn("other_language_title", F.when(
            (df.title_language != "zh") & (df.title_language != "zh_Hant") & (df.title_language != "en") &
            (df.title_language != "ja") & (df.title_language != "ko"), df.title))
        .collect()
        )
        print 'get several data',len(rows)
        samples.append(rows)
    print 'sample size is : {}'.format(len(sample))




def es_doc(review_id):
    es_connection = Elasticsearch(["http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200"],http_auth=("bdp","C38vEJEuraCw"),port=19200)
    doc = es_connection.search(index="int-ss-advancedreview_v1-apple-store-ios-all*", body={"query":{"match":{"_id":"{}".format(review_id)}}})
    return doc



def compare(doc, row):
    keys = [ u'_identifier', u'app_id', u'chinese_content', u'chinese_reply', u'chinese_title', u'content', u'content_language', u'country_code', u'date', u'device_code', u'english_content', u'english_reply', u'english_title', u'japanese_content', u'japanese_reply', u'japanese_title', u'korean_content', u'korean_reply', u'korean_title', u'language', u'market_code', u'other_language_content', u'other_language_reply', u'other_language_title', u'product_version', u'rating', u'reply', u'reply_date', u'reply_language', u'title', u'title_language', u'user_device', u'user_id', u'user_language', u'user_name', u'user_purchased', u'topic_ids']
    row_dict =  row
    print 'row dict', row_dict
     
    doc = doc["hits"]["hits"][0]["_source"]
    print doc
    for key in keys:
        if key in ("app_id", "_identifier"):
            assert str(doc[key]) == str(row_dict[key]), "{} is diffrent".format(key)
        else:
            assert doc[key] == row_dict[key], "{} is diffrent".format(key)

test_detail_of_unified_db(path_list)
for sample in samples:
    doc = es_doc(sample[0].review_id)
    compare(doc, sample[0])
    print 'pass'

In [0]:

import unittest
import boto3
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3 
from aadatapipelinecore.core.fs import Conf
import json
import time
from datetime import datetime
from elasticsearch import Elasticsearch
import json


# get up level data set

s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-advancedreview'))
# path_list= s3_bucket_list.all(prefix="unified/aa.adv_topic_201907_20200219/", depth_is_1=True )
path_list= s3_bucket_list.all(prefix="unified/aa.adv_join_20200218_20200221_1/", depth_is_1=True )
print path_list[0].split("=")[1].replace("-","").strip("/")

def es_count(month):
    es_connection = Elasticsearch(["http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200"],http_auth=("bdp","C38vEJEuraCw"),port=19200)
    doc = es_connection.count(index="int-ss-advancedreview_v1-*-{}".format(month))
    return int(doc["count"])

def test_compare_unified_db_count(path_list):
    for path in path_list:
        unified_count = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/{}".format(path)).select("review_id").distinct().count()
        month = path.split("=")[1].replace("-","").strip("/")
        db_count = es_count(month)
        if unified_count != db_count:
            print 'not equal!!! unified {} db {} month is {}'.format(unified_count, db_count, month)
        else:
            print 'pass!!! unified {} db {} month is {}'.format(unified_count, db_count, month)




# not equal!!! unified 43273 db 0 month is 201906
# not equal!!! unified 2108164 db 0 month is 201907
# not equal!!! unified 2013740 db 0 month is 201908
# not equal!!! unified 2004049 db 0 month is 201909
# not equal!!! unified 2277383 db 0 month is 201910
# not equal!!! unified 1998511 db 0 month is 201911
# not equal!!! unified 1970965 db 0 month is 201912
# not equal!!! unified 1901957 db 0 month is 202001
# not equal!!! unified 1068169 db 0 month is 202002
#  SPARK JOBS FINISHED   
# Took 1 min 3 sec. Last updated by fzhang at February 21 2020, 6:21:33 PM. (outdated)

test_compare_unified_db_count(path_list)

In [0]:
%%sh
PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132 -p 5432 -U citus_bdp_usage_qa -d aa_citus_db << EOF
SET search_path=advancedreview;

BEGIN;
-- SELECT count(*) FROM  advancedreview.advancedreview_topic_fact_v2 where country_code='US' and language='en' and array_length(topic_ids, 1)>=2 and topic_ids@>'{1}';
--SELECT * FROM  advancedreview.advancedreview_topic_fact_v2 where country_code='IN' and review_id='4395695885';
-- select * from advancedreview_topic_fact_v2 where  cardinality(topic_ids)=0 order by date desc limit 3;
-- select * from advancedreview_topic_fact_v2 where topic_ids <@ ARRAY[10]::smallint[] limit 10 ;
-- select _identifier, count(*) from advancedreview_topic_fact_v2 group by _identifier order by _identifier ;
select * from advancedreview_topic_fact_v3 order by date desc limit 3 ;

COMMIT;
EOF 


In [0]:

from pyspark.sql.functions import count, avg

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_not_join_20200218_20200221_1/").groupBy("event_month").agg({"event_month":"count"}).show()


In [0]:

from pyspark.sql.functions import count, avg

process_date_result = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview_post.topic.v1/").groupBy("process_date").agg({"process_date":"count"})
print  process_date_result.show()

process_date_result_list = process_date_result.collect()

print process_date_result_list[1]

In [0]:
%%sh
PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132 -p 5432 -U citus_bdp_usage_qa -d aa_citus_db << EOF
SET search_path=advancedreview;

BEGIN;
-- SELECT count(*) FROM  advancedreview.advancedreview_topic_fact_v2 where country_code='US' and language='en' and array_length(topic_ids, 1)>=2 and topic_ids@>'{1}';
--SELECT * FROM  advancedreview.advancedreview_topic_fact_v2 where country_code='IN' and review_id='4395695885';
-- select * from advancedreview_topic_fact_v2 where  cardinality(topic_ids)=0 order by date desc limit 3;
-- select * from advancedreview_topic_fact_v2 where topic_ids <@ ARRAY[10]::smallint[] limit 10 ;
-- select _identifier, count(*) from advancedreview_topic_fact_v2 group by _identifier order by _identifier ;
select count(*) from advancedreview_topic_fact_v3 where date between '2020-02-01' and '2020-02-22' ;

COMMIT;
EOF 

In [0]:
%%sh
curl -XGET -u bdp:C38vEJEuraCw -H 'Content-Type: application/json' http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/int-ss-advancedreview_v1-*202002/_stats?pretty

In [0]:

not_join_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_not_join_201908_20200211/").show()

not_join_df.groupBy("event_month").agg({"event_month":"count"}).show()

In [0]:



spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/").parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=*/process_date=2020-0{1-16,1-17,1-18,1-19,1-20,1-21,1-22,1-23,1-24,1-25,1-26,1-27,1-28,1-29,1-30,1-31,2*}/").registerTempTable("review_2020_01_16_")
val sql = (
    """
SELECT a.*,date_format(a.time, 'yyyy-MM') as event_month FROM
    review_2020_01_16_ a
    INNER JOIN (
        SELECT review_id,
        max(unix_timestamp(process_date, 'yyyy-MM-dd') + cast(process_hour as  INT)) AS process_time
        FROM review_2020_01_16_
        GROUP BY review_id
    ) b
    ON
        a.review_id = b.review_id
        AND unix_timestamp(a.process_date, 'yyyy-MM-dd') + cast(a.process_hour as  INT) = b.process_time
""")
val df = spark.sql(sql).cache()
df.registerTempTable("review_data")
df.write.option("compression", "gzip").partitionBy("event_month").parquet("s3://b2c-prod-data-pipeline-unified-review/unified/review_2020_01_16_2020_02_11_aggr")
spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview_post.topic.v1/").registerTempTable("topic_data")
val sql = ("""
    SELECT
        topic._identifier,
        date_format(topic.event_date, 'yyyy-MM') as event_month,
        topic.event_date as date,
        review.country_code,
        review.product_version,
        review.user_name,
        review.rating,
        review.language,
        review.market_code,
        review.device_code,
        review.user_device,
        review.user_purchased,
        review.user_id,
        review.app_id,
        review.user_language,
        review.title,
        review.title_language,
        CASE WHEN review.title_language == "ko" THEN review.title END AS korean_title,
        CASE WHEN review.title_language == "ja" THEN review.title END AS japanese_title,
        CASE WHEN review.title_language IN ("zh","zh_Hant") THEN review.title END AS chinese_title,
        CASE WHEN review.title_language == "en" THEN review.title END AS english_title,
        CASE WHEN review.title_language NOT IN ("zh","zh_Hant","en","ja", "ko") THEN review.title END AS other_language_title,
        review.content,
        review.content_language,
        CASE WHEN review.content_language == "ko" THEN review.content END AS korean_content,
        CASE WHEN review.content_language == "ja" THEN review.content END AS japanese_content,
        CASE WHEN review.content_language IN ("zh","zh_Hant") THEN review.content END AS chinese_content,
        CASE WHEN review.content_language == "en" THEN review.content END AS english_content,
        CASE WHEN review.content_language NOT IN ("zh","zh_Hant","en","ja", "ko") THEN review.content END AS other_language_content,
        review.reply,
        review.reply_language,
        CASE WHEN review.reply_language == "ko" THEN review.reply END AS korean_reply,
        CASE WHEN review.reply_language == "ja" THEN review.reply END AS japanese_reply,
        CASE WHEN review.reply_language IN ("zh","zh_Hant") THEN review.reply END AS chinese_reply,
        CASE WHEN review.reply_language == "en" THEN review.reply END AS english_reply,
        CASE WHEN review.reply_language NOT IN ("zh","zh_Hant","en","ja", "ko") THEN review.reply END AS other_language_reply,
        review.reply_date,
        topic.topic_ids,
        topic.review_id,
        review.review_id as review_id_
    FROM
        topic_data AS topic
        LEFT JOIN
        review_data AS review
        ON topic.review_id = review.review_id
""")
val df = spark.sql(sql).cache()
df.registerTempTable("joined_data")
spark.sql("select * from joined_data where review_id_ is null").write.option("compression", "gzip").partitionBy("event_month").parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/adv_join_202016_20200211_review_null")
spark.sql("select * from joined_data where review_id_ is not null").write.option("compression", "gzip").partitionBy("event_month").parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/adv_join_202016_20200211_review_is_not_null")
//2020-01-16 - 2020-02-11 process data review data join topic data 2.11 before
spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_join_201908_20200211/").registerTempTable("old_data")
spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/adv_join_202016_20200211_review_is_not_null").registerTempTable("new_data")
val sql1 = "select a.* from new_data a left join old_data b on a.review_id = b.review_id"
spark.sql(sql1).registerTempTable("new_review_")
val sql2 = "select b.* from new_data a right join old_data b on a.review_id = b.review_id where a.review_id is null"
spark.sql(sql2).registerTempTable("old_review_")
val columns = spark.sql("select * from old_review_").columns.mkString(",")
val df = spark.sql("select %s from new_review_ union all select %s from old_review_".format(columns,columns)).cache()
df.registerTempTable("joined_data")
df.write.option("compression", "gzip").mode("overwrite").partitionBy("event_month").parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_join_201908_20200211_2/")
spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview_post.topic.v1/").registerTempTable("topic_data")
(
    spark.sql("""select a.* ,b.review_id as review_id_ 
    from topic_data a left join joined_data b on a.review_id = b.review_id 
    where b.review_id is null""")
    .write
    .mode("overwrite")
    .option("compression", "gzip")
    .partitionBy("event_month")
    .parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/aa.adv_not_join_201908_20200211_2")
)